# AI算法工程师
## 毕业项目
### 猫狗识别

手动创建一个CNN模型以及使用预训练模型训练（迁移学习）。

### 1.加载数据
和上一个notebook中的创建流程一样，创建train_dataLoader和val_dataLoader

In [1]:
%load_ext autoreload
%autoreload 2
from model import CustomCNN, Pre_CNN_VGG

In [2]:
from dataloader import CatDogDataset
from torchvision import transforms
from utils import imagelist
import random
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch
import time
from pytorchtools import EarlyStopping


In [3]:
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          
    transforms.RandomCrop(224),                      
    transforms.RandomHorizontalFlip(),               
    transforms.ToTensor(),                          
    transforms.Normalize((0.485, 0.456, 0.406),      
                         (0.229, 0.224, 0.225))])
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])])

In [4]:
train_files = imagelist('data/train')
random.shuffle(train_files)
train_list = train_files[:20000]
valid_list = train_files[20000:]

In [5]:
train_dataset = CatDogDataset(image_paths=train_list, transform=transform_train, mode='train' )
val_dataset = CatDogDataset(image_paths=valid_list, transform=transform_val, mode='val' )


In [6]:
batch_size = 64
train_dataLoader = DataLoader(train_dataset, batch_size=batch_size)
val_dataLoader = DataLoader(val_dataset, batch_size=batch_size)

### 2.定义训练函数
可以通过给train_model传入不同的参数值来观察不同参数对训练结果的影响，方便调优。

In [9]:
def train_model(net, train_dataLoader, val_dataLoader, criterion, optimizer,early_stopping, num_epoch):
    train_loss_list = []
    train_accuracy_list = []
    val_loss_list = []
    val_accuracy_list = []
    start = time.time()
    net = net.cuda()
    for epoch in range(num_epoch):
        epoch_loss = 0
        epoch_accuracy = 0
        for X, y in train_dataLoader:
            X = X.cuda()
            y = y.cuda()
            preds = net(X)

            loss = criterion(preds, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            accuracy = ((preds.argmax(dim=1) == y).float().mean())
            epoch_accuracy += float(accuracy)
            epoch_loss += float(loss)
            #print('.', end='', flush=True)

        epoch_accuracy = epoch_accuracy/len(train_dataLoader)
        train_accuracy_list.append(epoch_accuracy)
        epoch_loss = epoch_loss / len(train_dataLoader)
        train_loss_list.append(epoch_loss)
        print("Epoch: {}, train loss: {:.4f}, train accracy: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))


        with torch.no_grad():
            val_epoch_loss = 0
            val_epoch_accuracy = 0
            for val_X, val_y in val_dataLoader:
                val_X = val_X.cuda()
                val_y = val_y.cuda()
                val_preds = net(val_X)
                val_loss = criterion(val_preds, val_y)

                val_epoch_loss += float(val_loss)
                val_accuracy = ((val_preds.argmax(dim=1) == val_y).float().mean())
                val_epoch_accuracy += float(val_accuracy)
            val_epoch_accuracy = val_epoch_accuracy/len(val_dataLoader)
            val_epoch_loss = val_epoch_loss / len(val_dataLoader)
            val_accuracy_list.append(val_epoch_accuracy)
            val_loss_list.append(val_epoch_loss)
            print("Epoch: {}, valid loss: {:.4f}, valid accracy: {:.4f}, time: {}\n".format(epoch, val_epoch_loss, val_epoch_accuracy, time.time() - start))
            
        early_stopping(val_epoch_loss, net)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    return train_loss_list ,train_accuracy_list ,val_loss_list ,val_accuracy_list

### 3.训练模型
##### 3.1使用自定义模型CustomCNN训练

In [10]:
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.02)
criterion = nn.CrossEntropyLoss()
num_epoch = 10
net = CustomCNN()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)
train_loss ,train_accuracy ,val_loss ,val_accuracy = train_model(net, train_dataLoader, val_dataLoader, criterion, optimizer,early_stopping, num_epoch)

Epoch: 0, train loss: 0.6120, train accracy: 0.6603, time: 155.69037652015686
Epoch: 0, valid loss: 0.5510, valid accracy: 0.7197, time: 192.28469395637512

Validation loss decreased (inf --> 0.551026).  Saving model ...
Epoch: 1, train loss: 0.5139, train accracy: 0.7515, time: 341.5398886203766
Epoch: 1, valid loss: 0.5147, valid accracy: 0.7532, time: 378.1076035499573

Validation loss decreased (0.551026 --> 0.514733).  Saving model ...
Epoch: 2, train loss: 0.4785, train accracy: 0.7738, time: 527.1865396499634
Epoch: 2, valid loss: 0.5043, valid accracy: 0.7516, time: 565.5379469394684

EarlyStopping counter: 1 out of 5
Epoch: 3, train loss: 0.4579, train accracy: 0.7884, time: 715.9470508098602
Epoch: 3, valid loss: 0.4513, valid accracy: 0.7933, time: 752.7274060249329

Validation loss decreased (0.514733 --> 0.451317).  Saving model ...
Epoch: 4, train loss: 0.4410, train accracy: 0.7977, time: 903.5906331539154
Epoch: 4, valid loss: 0.4398, valid accracy: 0.7979, time: 939.62

In [11]:
#保存模型
model_dir = 'saved_models/'
model_name = 'custom_model_1.pt'

torch.save(net.state_dict(), model_dir+model_name)

##### 3.2使用预训练VGG模型训练

In [12]:
early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.001)
criterion = nn.CrossEntropyLoss()
num_epoch = 20
net = Pre_CNN_VGG()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.001)
train_loss ,train_accuracy ,val_loss ,val_accuracy = train_model(net, train_dataLoader, val_dataLoader, criterion, optimizer,early_stopping, num_epoch)


Epoch: 0, train loss: 0.0911, train accracy: 0.9645, time: 241.2435019016266
Epoch: 0, valid loss: 0.0596, valid accracy: 0.9775, time: 303.01126623153687

Validation loss decreased (inf --> 0.059583).  Saving model ...
Epoch: 1, train loss: 0.0594, train accracy: 0.9777, time: 544.7107264995575
Epoch: 1, valid loss: 0.0510, valid accracy: 0.9777, time: 604.5309474468231

Validation loss decreased (0.059583 --> 0.050997).  Saving model ...
Epoch: 2, train loss: 0.0498, train accracy: 0.9816, time: 848.0505690574646
Epoch: 2, valid loss: 0.0518, valid accracy: 0.9800, time: 907.0242629051208

EarlyStopping counter: 1 out of 5
Epoch: 3, train loss: 0.0452, train accracy: 0.9835, time: 1148.5610387325287
Epoch: 3, valid loss: 0.0540, valid accracy: 0.9773, time: 1208.5379781723022

EarlyStopping counter: 2 out of 5
Epoch: 4, train loss: 0.0453, train accracy: 0.9827, time: 1449.6243815422058
Epoch: 4, valid loss: 0.0454, valid accracy: 0.9816, time: 1508.289235830307

Validation loss decr

In [9]:
model_dir = 'saved_models/'
model_name = 'pre_vgg_model_3.pt'

# after training, save your model parameters in the dir 'saved_models'
torch.save(net.state_dict(), model_dir+model_name)

可以看出，预训练的VGG模型效果更佳,因为VGG模型有更多的层，神经网络的层数对拟合效果的影响很大。

在pytorchtools.py文件中定义了earlystopping，在验证损失停止下降时结束训练，防止模型过拟合。

因为VGG模型有更复杂的结构，所以训练时的计算量大于自定义的简单模型，体现在训练时间上几乎是后者的二倍。

### 4.生成结果 csv 文件


In [22]:
from tqdm import tqdm
from PIL import Image
import torch.nn.functional as F
import pandas as pd

def predict(net,path):
    id_list = []
    pred_list = []
    test_list = imagelist('data/test')
    net.cuda()
    with torch.no_grad():
        for test_path in tqdm(test_list):
            img = Image.open(test_path)
            _id = int(test_path.split('/')[-1].split('.')[0])

            img = transform_val(img)
            img = img.unsqueeze(0)
            img = img.cuda()

            net.eval()

            outputs = net(img)
            preds = F.softmax(outputs, dim=1)[:, 1].tolist()

            id_list.append(_id)
            pred_list.append(preds[0])
        res = pd.DataFrame({'id': id_list, 'label': pred_list})
        res.sort_values(by='id', inplace=True)
        res.reset_index(drop=True, inplace=True)
        res.to_csv(path, index=False)

In [17]:
net = Pre_CNN_VGG()
predict(net,'vgg.csv')

100%|██████████| 12500/12500 [03:41<00:00, 56.53it/s]


In [23]:
net = CustomCNN()
predict(net,'CustomCNN.csv')

100%|██████████| 12500/12500 [02:20<00:00, 89.28it/s]
